1 - Automatiser l’interaction avec des pages cibles et le fil d’actualité :
- Liker les posts
- Commenter (en recopiant exactement la description du post) mais adaptable avec une API d'IA générative
- Republier le contenu

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains
import time, re

# ============ CONFIG ============
EMAIL = "email@gmail.com"
PASSWORD = "motDePasse"

TARGETS = [
    "https://www.linkedin.com/company/kisskissbankbank/",
    "https://www.linkedin.com/company/ulule/",
    "https://www.linkedin.com/in/romain-taugourdeau/"  # possible aussi pour un profil
]

MAX_POSTS = 2  # nombre de posts traités par cible
# ================================

def clean_text(s):
    return re.sub(r"\s+", " ", (s or "").strip())

def js_click(driver, el):
    driver.execute_script("arguments[0].scrollIntoView({block:'center'});", el)
    driver.execute_script("arguments[0].click();", el)

def insert_text_with_events(driver, el, text):
    # Insertion via JS (gère emojis + déclenche events)
    driver.execute_script("""
      const el = arguments[0], txt = arguments[1];
      el.focus();
      try {
        const sel = window.getSelection(), range = document.createRange();
        range.selectNodeContents(el); range.collapse(false);
        sel.removeAllRanges(); sel.addRange(range);
        document.execCommand('delete');
        document.execCommand('insertText', false, txt);
      } catch(e) {
        el.textContent = txt;
      }
      el.dispatchEvent(new InputEvent('input', {bubbles:true}));
      el.dispatchEvent(new Event('change', {bubbles:true}));
    """, el, text)

def get_full_description(post, driver):
    # 1) "Voir plus" si présent
    for xp in [
        ".//button[contains(.,'Voir plus') or contains(.,'See more')]",
        ".//span[contains(.,'Voir plus') or contains(.,'See more')]/ancestor::button[1]",
    ]:
        try:
            btn = post.find_element(By.XPATH, xp)
            js_click(driver, btn)
            time.sleep(0.4)
            break
        except: pass
    # 2) Récup texte
    for xp in [
        ".//div[contains(@class,'update-components-text')]",
        ".//div[contains(@class,'feed-shared-text')]",
        ".//span[contains(@class,'break-words')]",
        ".//*[@dir='ltr']",
    ]:
        try:
            raw = post.find_element(By.XPATH, xp).text
            if raw and raw.strip():
                return clean_text(raw)
        except: pass
    return ""

def try_repost(post, driver, wait):
    """Repost simple (sans commentaire). Retourne True si succès."""
    # Bouton Reposter / Repost / Partager (LinkedIn varie selon langue/expériences)
    repost_btn = None
    for xp in [
        ".//button[contains(@aria-label,'Reposter') or contains(@aria-label,'Repost') or contains(@aria-label,'Partager')]",
        ".//button[.//span[contains(.,'Reposter') or contains(.,'Repost') or contains(.,'Partager')]]",
    ]:
        try:
            repost_btn = post.find_element(By.XPATH, xp); break
        except: pass
    if not repost_btn:
        return False
    try:
        js_click(driver, repost_btn)
        time.sleep(0.6)
        # Menu avec 2 options: "Reposter" (simple) ou "Reposter avec votre avis"
        option = None
        for oxp in [
            "//span[normalize-space()='Reposter']/ancestor::button[1]",
            "//span[normalize-space()='Repost']/ancestor::button[1]",
            "//button[normalize-space()='Reposter']",
            "//button[normalize-space()='Repost']",
            # parfois, un bouton direct "Partager" suffit
            "//button[normalize-space()='Partager']",
        ]:
            try:
                option = driver.find_element(By.XPATH, oxp); break
            except: pass
        if option:
            js_click(driver, option)
            # attendre bandeau/toast (si dispo) ou petite pause
            time.sleep(0.8)
            return True
    except: pass
    return False

# -------- DÉMARRAGE CHROME --------
driver = webdriver.Chrome()
driver.maximize_window()
wait = WebDriverWait(driver, 20)

try:
    # -------- LOGIN AUTOMATIQUE --------
    driver.get("https://www.linkedin.com/login")
    wait.until(EC.presence_of_element_located((By.ID, "username"))).send_keys(EMAIL)
    driver.find_element(By.ID, "password").send_keys(PASSWORD)
    driver.find_element(By.ID, "password").send_keys(Keys.RETURN)
    wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))
    print("✅ Connexion envoyée.")
    print("⏳ Si un captcha/vérif apparaît, résous-le maintenant (attente 35s)…")
    time.sleep(35)

    # -------- BOUCLE SUR LES CIBLES --------
    for base in TARGETS:
        # Entreprise -> /posts/ ; Profil -> /recent-activity/all/
        target = base.rstrip("/") + ("/posts/" if "/company/" in base else "/recent-activity/all/")
        print(f"\n🔎 Cible: {target}")
        driver.get(target)
        wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))
        time.sleep(1.5)

        # Scroll léger
        for _ in range(2):
            driver.execute_script("window.scrollBy(0, 1200);")
            time.sleep(0.8)

        # Récup posts
        posts = driver.find_elements(By.TAG_NAME, "article")
        if not posts:
            posts = driver.find_elements(By.XPATH, "//div[contains(@data-urn,'urn:li:activity')]")
        if not posts:
            print("⚠️ Aucun post détecté.")
            continue

        for i, post in enumerate(posts[:MAX_POSTS], 1):
            print(f"\n--- Post {i} ---")

            # Description EXACTE
            desc = get_full_description(post, driver)
            if not desc:
                print("⚠️ Pas de description trouvée — on passe ce post.")
                continue

            # Like
            try:
                like_btn = post.find_element(
                    By.XPATH, ".//button[contains(@aria-label,'J’aime') or contains(@aria-label,'Like')]"
                )
                js_click(driver, like_btn)
                print("👍 Liké")
                time.sleep(0.6)
            except:
                print("⚠️ Pas de bouton Like (ou déjà liké)")

            # Commenter = description exacte
            try:
                # ouvrir éditeur
                try:
                    cbtn = post.find_element(
                        By.XPATH, ".//button[contains(@aria-label,'Commenter') or contains(@aria-label,'Comment')]"
                    )
                    js_click(driver, cbtn); time.sleep(0.6)
                except: pass

                # éditeur le plus proche du post
                editor = None
                for xp in [
                    ".//div[contains(@class,'comments-comment-box__editor')]//div[@contenteditable='true' and @role='textbox']",
                    ".//div[@contenteditable='true' and @role='textbox']",
                ]:
                    try:
                        editor = post.find_element(By.XPATH, xp); break
                    except: pass
                if not editor:
                    editor = wait.until(EC.presence_of_element_located(
                        (By.XPATH, "//div[@contenteditable='true' and @role='textbox']"))
                    )

                insert_text_with_events(driver, editor, desc)
                time.sleep(0.6)

                # publier
                publish = None
                for xp in [
                    ".//button[contains(@class,'comments-comment-box__submit-button') and not(contains(@class,'artdeco-button--disabled'))]",
                    ".//button[not(@disabled) and (contains(.,'Commenter') or contains(.,'Post'))]",
                ]:
                    try:
                        publish = post.find_element(By.XPATH, xp); break
                    except: pass
                if not publish:
                    publish = wait.until(EC.element_to_be_clickable((
                        By.XPATH, "//button[contains(@class,'comments-comment-box__submit-button') and not(@disabled)]"
                    )))
                ActionChains(driver).move_to_element(publish).pause(0.2).click().perform()
                print("💬 Commentaire posté")
                time.sleep(0.9)
            except Exception as e:
                print("⚠️ Impossible de commenter :", e)

            # Repost simple (optionnel)
            try:
                if try_repost(post, driver, wait):
                    print("🔁 Repost effectué")
                else:
                    print("ℹ️ Repost non disponible/échoué (UI variable).")
            except:
                print("ℹ️ Repost non disponible/échoué.")

    print("\n✅ Terminé.")

finally:
    driver.quit()

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains
import time, re

# ============ CONFIG ============
EMAIL = "email@gmail.com"
PASSWORD = "motDePasse*"

MAX_POSTS = 5  # nombre de posts du fil d’actualité à traiter
# ================================

def clean_text(s):
    return re.sub(r"\s+", " ", (s or "").strip())

def js_click(driver, el):
    driver.execute_script("arguments[0].scrollIntoView({block:'center'});", el)
    driver.execute_script("arguments[0].click();", el)

def insert_text_with_events(driver, el, text):
    driver.execute_script("""
      const el = arguments[0], txt = arguments[1];
      el.focus();
      try {
        const sel = window.getSelection(), range = document.createRange();
        range.selectNodeContents(el); range.collapse(false);
        sel.removeAllRanges(); sel.addRange(range);
        document.execCommand('delete');
        document.execCommand('insertText', false, txt);
      } catch(e) {
        el.textContent = txt;
      }
      el.dispatchEvent(new InputEvent('input', {bubbles:true}));
      el.dispatchEvent(new Event('change', {bubbles:true}));
    """, el, text)

def get_full_description(post, driver):
    # bouton "Voir plus" éventuel
    for xp in [
        ".//button[contains(.,'Voir plus') or contains(.,'See more')]",
        ".//span[contains(.,'Voir plus') or contains(.,'See more')]/ancestor::button[1]",
    ]:
        try:
            btn = post.find_element(By.XPATH, xp)
            js_click(driver, btn); time.sleep(0.4); break
        except: pass
    # description
    for xp in [
        ".//div[contains(@class,'update-components-text')]",
        ".//div[contains(@class,'feed-shared-text')]",
        ".//span[contains(@class,'break-words')]",
        ".//*[@dir='ltr']",
    ]:
        try:
            raw = post.find_element(By.XPATH, xp).text
            if raw and raw.strip():
                return clean_text(raw)
        except: pass
    return ""

def try_repost(post, driver):
    repost_btn = None
    for xp in [
        ".//button[contains(@aria-label,'Reposter') or contains(@aria-label,'Repost') or contains(@aria-label,'Partager')]",
        ".//button[.//span[contains(.,'Reposter') or contains(.,'Repost') or contains(.,'Partager')]]",
    ]:
        try:
            repost_btn = post.find_element(By.XPATH, xp); break
        except: pass
    if not repost_btn:
        return False
    try:
        js_click(driver, repost_btn); time.sleep(0.6)
        for oxp in [
            "//span[normalize-space()='Reposter']/ancestor::button[1]",
            "//span[normalize-space()='Repost']/ancestor::button[1]",
            "//button[normalize-space()='Reposter']",
            "//button[normalize-space()='Repost']",
            "//button[normalize-space()='Partager']",
        ]:
            try:
                option = driver.find_element(By.XPATH, oxp)
                js_click(driver, option); time.sleep(0.8)
                return True
            except: pass
    except: pass
    return False

# -------- MAIN --------
driver = webdriver.Chrome()
driver.maximize_window()
wait = WebDriverWait(driver, 20)

try:
    # LOGIN
    driver.get("https://www.linkedin.com/login")
    wait.until(EC.presence_of_element_located((By.ID, "username"))).send_keys(EMAIL)
    driver.find_element(By.ID, "password").send_keys(PASSWORD)
    driver.find_element(By.ID, "password").send_keys(Keys.RETURN)
    wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))
    print("✅ Connecté.")
    print("⏳ Si captcha apparaît, résous-le (35s).")
    time.sleep(35)

    # Aller à l’accueil
    driver.get("https://www.linkedin.com/feed/")
    wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))
    time.sleep(2)

    # Scroll pour charger plus de posts
    for _ in range(3):
        driver.execute_script("window.scrollBy(0, 1600);")
        time.sleep(1)

    # Récupération des posts
    posts = driver.find_elements(By.TAG_NAME, "article")
    if not posts:
        posts = driver.find_elements(By.XPATH, "//div[contains(@data-urn,'urn:li:activity')]")
    print(f"🔎 {len(posts)} posts détectés dans le flux.")

    for i, post in enumerate(posts[:MAX_POSTS], 1):
        print(f"\n--- Post {i} ---")
        desc = get_full_description(post, driver)
        if not desc:
            print("⚠️ Pas de description"); continue

        # Like
        try:
            like_btn = post.find_element(By.XPATH, ".//button[contains(@aria-label,'J’aime') or contains(@aria-label,'Like')]")
            js_click(driver, like_btn); print("👍 Liké"); time.sleep(0.6)
        except: print("⚠️ Pas de bouton Like")

        # Commenter
        try:
            try:
                cbtn = post.find_element(By.XPATH, ".//button[contains(@aria-label,'Commenter') or contains(@aria-label,'Comment')]")
                js_click(driver, cbtn); time.sleep(0.6)
            except: pass
            editor = post.find_element(By.XPATH, ".//div[@contenteditable='true' and @role='textbox']")
            insert_text_with_events(driver, editor, desc)
            time.sleep(0.6)
            publish = post.find_element(By.XPATH, ".//button[contains(@class,'comments-comment-box__submit-button') and not(@disabled)]")
            ActionChains(driver).move_to_element(publish).pause(0.2).click().perform()
            print("💬 Commentaire posté"); time.sleep(0.9)
        except Exception as e:
            print("⚠️ Impossible de commenter :", e)

        # Repost
        if try_repost(post, driver):
            print("🔁 Repost effectué")
        else:
            print("ℹ️ Repost non disponible")

    print("\n✅ Terminé.")

finally:
    driver.quit()

2 – Candidature automatique (Easy Apply)
- Parcourir les offres “Data Analyst” (filtrées Candidature simplifiée), ouvrir chaque annonce, enchaîner les étapes du formulaire et cliquer sur Envoyer la candidature, jusqu’à un quota donné (pages × offres).

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
import time, re

# ========= CONFIG =========
EMAIL = "email@gmail.com"
PASSWORD = "motDePasse"

ROLE_KEYWORDS = "Data Analyst"
LOCATION = "France"      # "" pour ne pas filtrer le lieu
MAX_APPLICATIONS = 10
PAGES_TO_BROWSE = 3
WAIT_AFTER_LOGIN = 30    # temps pour résoudre captcha si affiché
# =========================

def wait_body(d, t=15): WebDriverWait(d, t).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
def norm(s): return re.sub(r"\s+", " ", (s or "")).strip().lower()

def js_click(d, el):
    d.execute_script("arguments[0].scrollIntoView({block:'center'});", el)
    try: el.click()
    except: d.execute_script("arguments[0].click();", el)

def login(d):
    d.get("https://www.linkedin.com/login")
    wait_body(d)
    WebDriverWait(d, 15).until(EC.presence_of_element_located((By.ID, "username"))).send_keys(EMAIL)
    d.find_element(By.ID, "password").send_keys(PASSWORD)
    d.find_element(By.ID, "password").send_keys(Keys.RETURN)
    wait_body(d)
    print("✅ Connexion envoyée. Si captcha/vérif → fais-le maintenant…")
    time.sleep(WAIT_AFTER_LOGIN)

def open_search(d):
    d.get("https://www.linkedin.com/jobs/")
    wait_body(d); time.sleep(1)
    kw = WebDriverWait(d, 15).until(EC.presence_of_element_located((By.XPATH, "//input[starts-with(@id,'jobs-search-box-keyword-id-')]")))
    kw.clear(); kw.send_keys(ROLE_KEYWORDS)
    if LOCATION:
        try:
            loc = d.find_element(By.XPATH, "//input[starts-with(@id,'jobs-search-box-location-id-')]")
            loc.clear(); loc.send_keys(LOCATION)
        except: pass
    kw.send_keys(Keys.RETURN)
    wait_body(d); time.sleep(2)

    # Filtre Easy Apply
    try:
        allf = WebDriverWait(d, 8).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(.,'Tous les filtres') or contains(.,'All filters')]")))
        js_click(d, allf); time.sleep(1)
        chk = WebDriverWait(d, 8).until(EC.presence_of_element_located((By.XPATH, "//label[contains(.,'Candidature simplifiée') or contains(.,'Easy Apply')]//input[@type='checkbox']")))
        d.execute_script("arguments[0].click();", chk)
        apply = WebDriverWait(d, 8).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(.,'Afficher les offres') or contains(.,'Show results')]")))
        js_click(d, apply); time.sleep(2)
    except:
        print("ℹ️ Filtre Easy Apply déjà actif ou UI différente.")

def collect_links_on_page(d, want=40):
    for _ in range(3):
        d.execute_script("window.scrollBy(0, 1600);"); time.sleep(0.7)
    cards = d.find_elements(By.CSS_SELECTOR, "ul.scaffold-layout__list-container li a.job-card-container__link")
    if not cards:
        cards = d.find_elements(By.CSS_SELECTOR, "a[data-control-name='job_card']")
    links = []
    for a in cards:
        href = a.get_attribute("href")
        if href and "linkedin.com/jobs/view/" in href:
            links.append(href.split("?")[0])
        if len(links) >= want: break
    return links

def go_next_page(d):
    for xp in [
        "//button[contains(@aria-label,'Suivant') or contains(@aria-label,'Next')]",
        "//button[contains(.,'Suivant') or contains(.,'Next')]",
    ]:
        try:
            btn = d.find_element(By.XPATH, xp)
            js_click(d, btn); wait_body(d); time.sleep(1.3)
            return True
        except: pass
    return False

def advance_modal_until_submit_and_send(d):
    """
    Enchaîne Suivant/Review et clique 'Envoyer la candidature' quand dispo.
    Retourne True si l’envoi a été tenté/cliqué.
    """
    for _ in range(12):  # limite défensive d’étapes
        # Submit visible ?
        for xp in [
            "//button[not(@disabled) and (contains(.,'Envoyer la candidature') or contains(.,'Submit application'))]",
            "//button[contains(@aria-label,'Envoyer la candidature') or contains(@aria-label,'Submit application')]",
        ]:
            try:
                submit = d.find_element(By.XPATH, xp)
                d.execute_script("arguments[0].style.outline='3px solid #2ecc71';", submit)
                js_click(d, submit); time.sleep(1.5)
                return True
            except: pass

        # sinon Review / Next
        clicked = False
        for xp in [
            "//button[not(@disabled) and (contains(.,'Révision') or contains(.,'Review'))]",
            "//button[contains(@aria-label,'Review') or contains(@aria-label,'Révision')]",
            "//button[not(@disabled) and (contains(.,'Suivant') or contains(.,'Next'))]",
            "//button[contains(@aria-label,'Suivant') or contains(@aria-label,'Next')]",
        ]:
            try:
                btn = d.find_element(By.XPATH, xp)
                js_click(d, btn); time.sleep(1.0)
                clicked = True; break
            except: pass

        if not clicked:
            # Peut-être des champs requis visibles : tenter de les auto-remplir avec valeurs déjà présentes
            try:
                # Cliquer toutes les “Continuer sans CV” / “Suivant” alternatives si présentes
                for alt in d.find_elements(By.XPATH, "//button[not(@disabled) and (contains(.,'Continuer') or contains(.,'Continue'))]"):
                    js_click(d, alt); time.sleep(0.8)
                    clicked = True
            except: pass

            if not clicked:
                return False
    return False

def easy_apply_flow(d):
    # bouton Easy Apply
    btn = None
    for xp in [
        "//button[contains(.,'Candidature simplifiée') or contains(.,'Easy Apply')]",
        "//button[contains(.,'Postuler') and contains(@class,'jobs-apply-button')]",
    ]:
        try:
            btn = d.find_element(By.XPATH, xp); break
        except: pass
    if not btn:
        print("— Pas de Easy Apply pour cette offre.")
        return False

    js_click(d, btn); time.sleep(1.2)
    sent = advance_modal_until_submit_and_send(d)

    # Fermer éventuellement la modale de confirmation / toast (si besoin)
    try:
        time.sleep(1.0)
        for xp in [
            "//button[contains(.,'Terminé') or contains(.,'Done')]",
            "//button[contains(.,'Fermer') or contains(.,'Close')]",
            "//button[contains(@aria-label,'Fermer') or contains(@aria-label,'Close')]",
        ]:
            try:
                close = d.find_element(By.XPATH, xp)
                js_click(d, close); time.sleep(0.6); break
            except: pass
    except: pass

    return sent

# ------------- MAIN -------------
opts = webdriver.ChromeOptions()
# Astuce conseillée : réutiliser un profil pour éviter captcha à chaque run
# opts.add_argument(r"--user-data-dir=C:\Users\TONUSER\AppData\Local\Google\Chrome\User Data\LinkedInApplyBot")

driver = webdriver.Chrome(options=opts)
driver.maximize_window()

try:
    login(driver)
    open_search(driver)

    prepared = 0
    page = 1

    while prepared < MAX_APPLICATIONS and page <= PAGES_TO_BROWSE:
        links = collect_links_on_page(driver, want=50)
        print(f"🗂️ Offres détectées sur la page {page}: {len(links)}")
        if not links:
            break

        for url in links:
            if prepared >= MAX_APPLICATIONS:
                break
            driver.get(url)
            wait_body(driver); time.sleep(1.0)

            ok = easy_apply_flow(driver)
            if ok:
                prepared += 1
                print(f"✅ Candidature envoyée (tentative) ({prepared}/{MAX_APPLICATIONS}) — {url}")
                time.sleep(1.2)
            else:
                print(f"⏭️ Offre ignorée (pas Easy Apply ou flow bloqué) — {url}")

        if prepared < MAX_APPLICATIONS:
            if not go_next_page(driver):
                print("ℹ️ Plus de pages ou navigation suivante indisponible.")
                break
            page += 1

    print(f"\n🎯 Terminé. Candidatures tentées : {prepared}")

finally:
    pass
    # driver.quit()

3 – Prospection (Ajout de contacts + note automatique) depuis Accueil / Mon réseau, parcourir les cartes “Se connecter”, puis pour chacune :

- Cliquer “Se connecter”,
- Ajouter une note (message court ≤300 caractères),
- Envoyer l’invitation,
- Répéter en boucle jusqu’à N invitations (avec pauses et rafraîchissements pour rester humain).

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time, random

# ====== CONFIG ======
EMAIL = "ton.email@exemple.com"
PASSWORD = "tonMotDePasseIci"

TARGET_INVITES = 15             # nombre total d'invitations à envoyer
AFTER_LOGIN_WAIT = 35           # marge pour résoudre un éventuel puzzle/captcha
REFRESH_DELAY = (5, 9)          # pause entre rafraîchissements
CARD_PAUSE    = (1.0, 2.0)      # délai entre personnes (human-like)
NOTE_TEXT = ("Bonjour,\n"
             "Je suis data analyst junior. Est-ce que votre entreprise embauche "
             "ou ouvre des stages/alternances en data ? Heureux de connecter !")
# ====================

def nap(a=0.8, b=1.6): time.sleep(random.uniform(a, b))
def wait_body(d, t=15): WebDriverWait(d, t).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
def js_click(d, el):
    d.execute_script("arguments[0].scrollIntoView({block:'center'});", el)
    try: el.click()
    except: d.execute_script("arguments[0].click();", el)

def insert_text(d, el, text):
    d.execute_script("""
      const el = arguments[0], txt = arguments[1];
      el.focus();
      try {
        const sel = window.getSelection(), range = document.createRange();
        range.selectNodeContents(el); range.collapse(false);
        sel.removeAllRanges(); sel.addRange(range);
        document.execCommand('delete');
        document.execCommand('insertText', false, txt);
      } catch(e) {
        el.value = txt; el.dispatchEvent(new Event('input', {bubbles:true}));
      }
    """, el, text)

def login(d):
    d.get("https://www.linkedin.com/login")
    wait_body(d)
    WebDriverWait(d, 15).until(EC.presence_of_element_located((By.ID, "username"))).send_keys(EMAIL)
    d.find_element(By.ID, "password").send_keys(PASSWORD)
    d.find_element(By.ID, "password").send_keys(Keys.RETURN)
    wait_body(d)
    print("✅ Connexion envoyée. Si LinkedIn montre un puzzle, résous-le maintenant…")
    time.sleep(AFTER_LOGIN_WAIT)

def get_connect_buttons_on_feed(d):
    # cible le panneau “Personnes que vous pourriez connaître”
    buttons = []
    try:
        side = d.find_element(
            By.XPATH,
            "//aside//*[contains(normalize-space(.),'Personnes que vous pourriez connaître') or contains(normalize-space(.),'People you may know')]/ancestor::aside[1]"
        )
        buttons = side.find_elements(By.XPATH, ".//button[normalize-space()='Se connecter' or normalize-space()='Connect']")
    except:
        pass
    if not buttons:
        # fallback global
        buttons = d.find_elements(By.XPATH, "//button[normalize-space()='Se connecter' or normalize-space()='Connect']")
    return buttons

def send_invite_from_button(d, btn):
    # 1) Se connecter
    try:
        js_click(d, btn); nap()
    except:
        return False

    # 2) Ajouter une note (ou confirmation puis note)
    add = None
    for xp in [
        "//button[normalize-space()='Ajouter une note']",
        "//button[normalize-space()='Add a note']",
    ]:
        try:
            add = d.find_element(By.XPATH, xp); break
        except: pass

    if not add:
        # Parfois il faut confirmer “Se connecter” d'abord
        for xp in ["//button[normalize-space()='Se connecter']", "//button[normalize-space()='Connect']"]:
            try:
                js_click(d, d.find_element(By.XPATH, xp)); nap(); break
            except: pass
        for xp in [
            "//button[normalize-space()='Ajouter une note']",
            "//button[normalize-space()='Add a note']",
        ]:
            try:
                add = d.find_element(By.XPATH, xp); break
            except: pass
    if not add:
        return False

    js_click(d, add); nap()

    # 3) Saisir la note (300c max)
    ta = None
    for xp in ["//textarea[@name='message']", "//textarea[contains(@id,'custom-message')]", "//textarea"]:
        try:
            ta = d.find_element(By.XPATH, xp); break
        except: pass
    if not ta: 
        return False
    insert_text(d, ta, NOTE_TEXT[:290]); nap()

    # 4) Envoyer (auto)
    for xp in ["//button[normalize-space()='Envoyer']", "//button[normalize-space()='Send']"]:
        try:
            send = d.find_element(By.XPATH, xp)
            js_click(d, send); nap()
            return True
        except: pass
    return False

# ------------- MAIN -------------
opts = webdriver.ChromeOptions()
# CONSEILLÉ : réutiliser un profil Chrome pour limiter les captchas (décommente, adapte le chemin Windows/Mac/Linux)
# opts.add_argument(r"--user-data-dir=C:\Users\TONUSER\AppData\Local\Google\Chrome\User Data\LinkedInProspects")
driver = webdriver.Chrome(options=opts)
driver.maximize_window()

try:
    login(driver)

    sent = 0
    while sent < TARGET_INVITES:
        # Aller à l’accueil
        driver.get("https://www.linkedin.com/feed/")
        wait_body(driver); time.sleep(1.5)

        btns = get_connect_buttons_on_feed(driver)
        print(f"🔎 Boutons 'Se connecter' trouvés : {len(btns)}")

        # Si rien trouvé, tente “Mon réseau”
        if not btns:
            driver.get("https://www.linkedin.com/mynetwork/")
            wait_body(driver); time.sleep(1.5)
            btns = driver.find_elements(By.XPATH, "//button[normalize-space()='Se connecter' or normalize-space()='Connect']")
            print(f"🔎 Sur 'Mon réseau' : {len(btns)} boutons")

        for b in btns:
            if sent >= TARGET_INVITES:
                break
            try: driver.execute_script("arguments[0].scrollIntoView({block:'center'});", b)
            except: pass

            ok = send_invite_from_button(driver, b)
            if ok:
                sent += 1
                print(f"✅ Invitation envoyée {sent}/{TARGET_INVITES}")
            else:
                print("⏭️ Échec/indisponible pour ce contact.")
            nap(*CARD_PAUSE)

        if sent < TARGET_INVITES:
            t = random.uniform(*REFRESH_DELAY)
            print(f"🔁 Rafraîchissement dans {t:.1f}s…")
            time.sleep(t)

    print(f"\n🎯 Terminé. Invitations envoyées : {sent}")
finally:
    pass
    # driver.quit()